<a href="https://www.kaggle.com/code/amitpant7/hardware-aware-nas-for-fpgas-evolutionary-algo?scriptVersionId=158042363" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Searching for Hardware Aware Neural Networks on ImageNet for FPGA in MobilenetV3 Search Space


We will be utilizing **[Once-for-All (OFA)](https://github.com/mit-han-lab/once-for-all)** network trained on MobilenetV3 search space as a our supernetwork to retrieve the weights and to estimate the accuracy of subnets in our search space because it requires significant amount of time to train each subnet for evaluating its perfomace. This slows down the searching process. Utilizing the OFA would speed up the search process as we won't have to train the network at all. The search is perfomed on the Imagenet Dataset. 

#### #To Do - Construct Latency Table and Add latency Constraint


## 1. Preparation
Let's first install all the required packages:

In [ ]:
%%script echo skipping  

print('Installing PyTorch...')
! pip install torch 
print('Installing torchvision...')
! pip install torchvision
print('Installing numpy...')
! pip install numpy 
# thop is a package for FLOPs computing.
print('Installing thop (FLOPs counter) ...')
! pip install thop 
# ofa is a package containing training code, pretrained specialized models and inference code for the once-for-all networks.
# print('Installing OFA...')
# ! pip install ofa 
# tqdm is a package for displaying a progress bar.
print('Installing tqdm (progress bar) ...')
! pip install tqdm 
print('Installing matplotlib...')
! pip install matplotlib 
!pip install graphviz
! pip install torch-summary 

print('All required packages have been successfully installed!')


In [ ]:
# For kaggle

# !rm -r /kaggle/working/*
!pip install thop 
! pip install gdown
!pip install shutil
!pip install graphviz
! pip install torch-summary 
! git clone https://github.com/amitpant7/Evolutionary-Neural-Architectural-Search-for-FPGAs.git
! mv /kaggle/working/Evolutionary-Neural-Architectural-Search-for-FPGAs/* /kaggle/working
! rm -r Evolutionary-Neural-Architectural-Search-for-FPGAs


Then, we can import the packages used in this tutorial:

In [ ]:
import os
import torch
import torch.nn as nn
from torchvision import transforms, datasets
import numpy as np
import time
import random
import shutil
import math
from PIL import Image
import copy
from matplotlib import pyplot as plt
from torchsummary import summary

from ofa.model_zoo import ofa_net
from ofa.utils import download_url

from ofa.accuracy_predictor import AccuracyPredictor
from ofa.flops_table import ArthIntTable

from ofa.latency_table import LatencyTable
from ofa.evolution_finder import EvolutionFinder
from ofa.imagenet_eval_helper import evaluate_ofa_subnet, evaluate_ofa_specialized
from ofa.imagenet_classification.elastic_nn.networks.ofa_mbv3 import OFAMobileNetV3

from ofa.utils.arch_visualization_helper import draw_arch
# from ofa.tutorial import AccuracyPredictor, FLOPsTable, LatencyTable, EvolutionFinder
# from ofa.tutorial import evaluate_ofa_subnet, evaluate_ofa_specialized

from tqdm import tqdm

# set random seed
random_seed = 1
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
print('Successfully imported all packages and configured random seed to %d!'%random_seed)

Now it's time to determine which device to use for neural network inference in the rest of this tutorial. If your machine is equipped with GPU(s), we will use the GPU by default. Otherwise, we will use the CPU.

In [ ]:
#os.environ['CUDA_VISIBLE_DEVICES'] = '0'
cuda_available = torch.cuda.is_available()
if cuda_available:
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = True
    torch.cuda.manual_seed(random_seed)
    print('Using GPU.')
else:
    print('Using CPU.')

##  2. Architecutre Visualization & Encoding: Exploring the OFA network 

Good! Now you have successfully configured the environment! It's time to import the **OFA network** for the following experiments.
The OFA network used in this tutorial is built upon MobileNetV3 with width multiplier 1.2, supporting elastic depth (2, 3, 4) per stage, elastic expand ratio (3, 4, 6), and elastic kernel size (3, 5 7) per block.

In [ ]:
net_id  = 'ofa_mbv3_d234_e346_k357_w1.2'
url_base = "https://raw.githubusercontent.com/han-cai/files/master/ofa/ofa_nets/"

ofa_network = OFAMobileNetV3(
            dropout_rate=0,
            width_mult=1.2,
            ks_list=[3, 5, 7],
            expand_ratio_list=[3, 4, 6],
            depth_list=[2, 3, 4],
        )

pt_path = download_url(url_base + net_id, model_dir=".torch/ofa_nets")
init = torch.load(pt_path, map_location="cpu")["state_dict"]
ofa_network.load_state_dict(init)
print('Supernetwork Ready')

In [ ]:
# super_net_name = "ofa_supernet_mbv3_w12"
# super_net = torch.hub.load('mit-han-lab/once-for-all', super_net_name, pretrained=True)

Lets visualize a randomly sampled network from our supernet.

In the architecture visualization, the legend of each block MBConv{e}-{k}x{k} means that the current block is a mobile inverted block with expand ratio e and the kernel size of the depthwise convolution layer is k. Different colors of the blocks indicate different kernel sizes, and gray blocks are network stage dividers. Different widths for the blocks indicate different expand ratios. We also annotate the output resolution close to each block.

In [ ]:
# Randomly sample sub-networks from OFA network
image_size = 224

cfg1 = ofa_network.sample_active_subnet()
subnet = ofa_network.get_active_subnet(preserve_weight=True)

#manualy set the subnet 
cfg = ofa_network.set_active_subnet(ks=3, e=6, d=4)

cfg = ofa_network.set_max_net()
subnet2 = ofa_network.get_active_subnet(preserve_weight=True)

In [ ]:
def visualize_subnet(cfg):
    draw_arch(cfg["ks"], cfg["e"], cfg["d"], image_size, out_name="viz/subnet")
    im = Image.open("viz/subnet.png")
    im = im.rotate(90, expand=1)
    fig = plt.figure(figsize=(im.size[0] / 250, im.size[1] / 250))
    plt.axis("off")
    plt.imshow(im)
    plt.show()

visualize_subnet(cfg)

 
Every subnet in the ofa can be represented in the form of python dictionary. This encoding helps us to represent entire netowk with just few numbers.

Here is one expample of encoding

In [ ]:
print('The architecture encoding of random subnetwork', cfg)

## 3. Dataset Preperation

Now, let's build the ImageNet dataset and the corresponding dataloader. Notice that **if you are not in kaggle it will be skipped** since it will be very slow.


We will only use subset of ImageNet validation set which will contains 10,000 images for testing.


**I will utilize subsets of imagenet for both validation and retraining.**

In [ ]:
%%script echo skipping

def make_subset(old_path, new_path, number_per_class=10):
    # Remove existing new_path if it exists
    if os.path.exists(new_path):
        shutil.rmtree(new_path)

    os.makedirs(new_path)

    print('Creating subset...')
    dirs = [d for d in os.listdir(old_path) if os.path.isdir(os.path.join(old_path, d))]

    for i, directory in enumerate(dirs):
        directory_path = os.path.join(old_path, directory)
        filenames_in_dir = [filename for filename in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, filename))]
        sampled_filenames = random.sample(filenames_in_dir, min(number_per_class, len(filenames_in_dir)))

        new_directory_path = os.path.join(new_path, directory)
        os.makedirs(new_directory_path)

        for filename in sampled_filenames:
            src_file_path = os.path.join(directory_path, filename)
            tgt_file_path = os.path.join(new_directory_path, filename)

            try:
                shutil.copy(src_file_path, tgt_file_path)
            except Exception as e:
                print(f"Error copying {src_file_path} to {tgt_file_path}: {e}")
                
    print('Subset creation complete.')

# Example usage
old_val = '/kaggle/input/imagenet1kvalidation/val'
new_val = '/kaggle/working/imagenet_sub_val'

make_subset(old_val, new_val, number_per_class=10)

In [ ]:
batch_size=32

#I will use a susbset of imagenetval of 10k images 
if cuda_available:
    # path to the ImageNet dataset
    # link --> https://www.kaggle.com/datasets/titericz/imagenet1k-val
    
    imagenet_data_path = '/kaggle/working/imagenet_sub_val'

    # if 'imagenet_data_path' is empty, download a subset of ImageNet containing 2000 images (~250M) for test
    if not os.path.isdir(imagenet_data_path):
        print('%s is empty. Download a subset of ImageNet for test.' % imagenet_data_path)

    print('The ImageNet dataset files are ready.')
else:
    print('Since GPU is not found in the environment, we skip all scripts related to ImageNet evaluation.')
    
    
  
if cuda_available:
    # The following function build the data transforms for test
    def build_val_transform(size):
        return transforms.Compose([
            transforms.Resize(int(math.ceil(size / 0.875))),
            transforms.CenterCrop(size),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            ),
        ])
    
    val_data = datasets.ImageFolder(
            root=os.path.join(imagenet_data_path),
            transform=build_val_transform(224)
        )
    

    val_loader = torch.utils.data.DataLoader(
        val_data,
        batch_size=batch_size,  
        shuffle = True,
        num_workers=4,  
        pin_memory=True,
        drop_last=False,
    )
    print('The ImageNet dataloader is ready. Size : {}'.format(len(val_loader)*batch_size))
else:
    data_loader = None
    print('Since GPU is not found in the environment, we skip all scripts related to ImageNet evaluation.')

Now you have configured the dataset. Let's build the dataloader for evaluation.
Again, this will be skipped if you are in a CPU environment.

Lets evaluate our randomly sampled network on imagenet validation set 

In [ ]:
from ofa.utils.common_tools import *
def evaluate_sub(net, data_loader=val_loader ,device="cuda:0"):
    if "cuda" in device:
        net = torch.nn.DataParallel(net).to(device)
    else:
        net = net.to(device)

    criterion = nn.CrossEntropyLoss().to(device)

    net.eval()
    net = net.to(device)
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    with torch.no_grad():
        with tqdm(total=len(data_loader), desc="Validate") as t:
            for i, (images, labels) in enumerate(data_loader):
                images, labels = images.to(device), labels.to(device)
                # compute output
                output = net(images)
                loss = criterion(output, labels)
                # measure accuracy and record loss
                acc1, acc5 = accuracy(output, labels, topk=(1, 5))

                losses.update(loss.item(), images.size(0))
                top1.update(acc1[0].item(), images.size(0))
                top5.update(acc5[0].item(), images.size(0))
                t.set_postfix(
                    {
                        "loss": losses.avg,
                        "top1": top1.avg,
                        "top5": top5.avg,
                        "img_size": images.size(2),
                    }
                )
                t.update(1)

    print(
        "Results: loss=%.5f,\t top1=%.1f,\t top5=%.1f"
        % (losses.avg, top1.avg, top5.avg)
    )
    return top1.avg

In [ ]:
%%script echo skipping
if cuda_available:
    top1 = evaluate_sub(subnet2)

    print('Finished evaluating the pretrained sub-network: %s!' % top1)
else:
    print('Since GPU is not found in the environment, we skip all scripts related to ImageNet evaluation.')

## 4. Accuracy Predictor 

The key components of very fast neural network deployment are **accuracy predictors** and **efficiency predictors**.
For the accuracy predictor, it predicts the Top-1 accuracy of a given sub-network on a **holdout validation set**
(different from the official 50K validation set) so that we do **NOT** need to run very costly inference on ImageNet
while searching for specialized models. Such an accuracy predictor is trained using an accuracy dataset built with the OFA network.

In [ ]:
# accuracy predictor
accuracy_predictor = AccuracyPredictor(
    pretrained=True,
    device='cuda:0' if cuda_available else 'cpu'
)

print('The accuracy predictor is ready!')
print(accuracy_predictor.model)

Lets try predicting accuracy of our randomly sampled subnet

In [ ]:
cfg['r']= [224]

acc = accuracy_predictor.predict_accuracy([cfg])
print(acc*100)

Now, we have the powerful **accuracy predictor**. We then introduce two types of **efficiency predictors**: the latency predictor and the FLOPs predictor. 

The intuition of having efficiency predictors, especially the latency predictor, is that measuring the latency of a sub-network on-the-fly is also costly, especially for FPGA devices becuase it takes hours of work to implement even a single network and then measure latency.

The latency predictor is designed to eliminate this cost.

## 5. Searching with Arthemetic Intensity Constraint
For edge deployement like FPGAs computation is cheap where as memory movement is expensive. That's why we need to do as much as computation with fewer number of data movement. This can be defined by arthemetic constraint which is the ratio of MACs to size in bytes. 

Now, let's proceed towards searching for efficient network under arthemetic intensity constraint. We use the same accuracy predictor since accuracy predictors are agnostic to the types of efficiency constraint . For the efficiency predictor, we a arthametic intensity table. You can run the code below to setup it in a few seconds.

In [ ]:
arthemetic_intensity_lookup = ArthIntTable(pred_type='arthemetic_intensity', 
                                  device='cuda:0' if cuda_available else 'cpu',batch_size=1, 
                                  )

print('The Arthemetic Intensity lookup table is ready!')

Lets try to find the Arthemetic intensity of our random sampled subnet

In [ ]:
arth = 1/arthemetic_intensity_lookup.predict_efficiency(cfg)   #as it returns bytes/ops
print('arthemetic intensity = {} ops/byte'.format(arth))

### Run Evolutionary search

In [ ]:
#  Hyper-parameters for the evolutionary search process

P = 2500  # The size of population in each generation
N = 1000  # How many generations of population to be searched
r = 0.25  # The ratio of networks that are used as parents for next generation

params = {
    'constraint_type': 'arthemetic_intensity', # Let's do FLOPs-constrained search
    'efficiency_constraint': 13,  # ai constraint , suggested range [10, 25]
    'mutate_prob': 0.1, # The probability of mutation in evolutionary search
    'mutation_ratio': 0.5, # The ratio of networks that are generated through mutation in generation n >= 2.
    'efficiency_predictor': arthemetic_intensity_lookup, # To use a predefined efficiency predictor.
    'accuracy_predictor': accuracy_predictor, # To use a predefined accuracy_predictor predictor.
    'population_size': P,
    'max_time_budget': N,
    'parent_ratio': r,
}


finder = EvolutionFinder(**params)


result_lis = []
result_valids = []
info = []
for arth_intensity in [15,18, 20]:
    st = time.time()
    finder.set_efficiency_constraint(arth_intensity)
    best_valids, best_info = finder.run_evolution_search()
    ed = time.time()
    
    print('Found best architecture at arthemetic intensity >= %.2f M in %.2f seconds! It achieves %.2f%s predicted accuracy with arthemetic intensity of %.2f ops/byte./n' % (arth_intensity, ed-st, best_info[0] * 100, '%',1/ best_info[-1]))
    result_lis.append(best_info)
    result_valids.append(best_valids)
    info.append(ed-st)

Lets visualize and evalute the found architecutre

In [ ]:
result_lis

In [ ]:
result_lis = [(0.8211345982551575,
  {'wid': None,
   'ks': [5, 5, 5, 5, 7, 3, 7, 3, 3, 7, 5, 5, 7, 7, 5, 7, 7, 3, 5, 5],
   'e': [4, 6, 6, 3, 6, 6, 4, 4, 4, 3, 4, 6, 6, 6, 6, 6, 6, 6, 6, 4],
   'd': [4, 4, 4, 4, 4],
   'r': [224]},
  0.06222183684342464),
 (0.8499683141708374,
  {'wid': None,
   'ks': [7, 5, 7, 5, 7, 5, 7, 7, 5, 7, 5, 7, 5, 7, 7, 3, 7, 3, 5, 5],
   'e': [6, 6, 6, 4, 6, 6, 4, 6, 6, 6, 4, 6, 6, 6, 6, 6, 6, 6, 6, 4],
   'd': [4, 4, 4, 4, 4],
   'r': [224]},
  0.05551231282144039),
 (0.8482525944709778,
  {'wid': None,
   'ks': [7, 5, 7, 5, 7, 7, 7, 7, 5, 5, 7, 7, 7, 7, 5, 7, 5, 3, 7, 7],
   'e': [6, 6, 6, 4, 6, 6, 6, 6, 6, 6, 4, 6, 6, 6, 6, 6, 6, 4, 4, 4],
   'd': [4, 4, 4, 4, 4],
   'r': [224]},
  0.049792223561726204)]

Evaluation of the searched networks on imagnet10k

In [ ]:
for arch in result_lis:
    cfg = arch[1]
    visualize_subnet(cfg)  
    ofa_network.set_active_subnet(cfg['ks'], cfg['e'], cfg['d'])
    network = ofa_network.get_active_subnet()
    acc = evaluate_sub(network)
    
    print('The evaluated accuracy is : {}'.format(acc))
    print('--'*50)

In [ ]:
archs = [
['Efficientnet B0', 77.8, 19.5],
['Mobilenet V3_large', 77, 11.5],
['nvidia_efficientnet_b4',82, 20.3]
]

In [ ]:

y = [res[0]*100 for res in result_lis]+ [79.5] #added evaulated accuracy after one epcoh train
x = [1/res[2] for res in result_lis]+ [20.1]



plt.figure(figsize=(12, 8))
plt.xlabel('Arithmetic Intensity (ops/byte)', fontsize=14)
plt.ylabel('Predicted Accuracy', fontsize=14)

# Plot the line and scatter plot with 'X' marker for each point in the same color
plt.plot(x, y, marker='X', linestyle='-', color='blue', markersize=8, label = 'Ours')
plt.plot(archs[0][2], archs[0][1], marker='*', color='red', label ='other', markersize=10 )
plt.plot(archs[1][2],archs[1][1], marker = '*', color = 'red', markersize = 10)
plt.plot(archs[2][2],archs[2][1], marker = '*', color = 'red', markersize = 10)


# Annotate the middle point
plt.text(20.2,79, f' Evaluated Accuracy \n after 1 epoch', fontsize = 10)
plt.text(17, 85.2, f'      FPGA specialized networks', fontsize=12, verticalalignment='bottom', horizontalalignment='left', color='blue')
plt.text(archs[0][2], archs[0][1], f'  Efficientnet_b0', fontsize=12 )
plt.text(archs[1][2],archs[1][1], f' {archs[1][0]}')
plt.text(archs[2][2]+0.1,archs[2][1], f' {archs[2][0]}', fontsize=12)

plt.ylim(75, 89)
plt.xlim(10, 24)
# Add legend
plt.legend(fontsize=12)
plt.title('Searching for FPGAs')
# Show the plot
plt.show()

## 5. Retrain the network
Lets fine tune the network on a subset of training set of imagenet.  
We will retrain the third searched model with ops/byte>20

In [ ]:
#extracting the model, 
cfg = result_lis[2][1]
ofa_network.set_active_subnet(cfg['ks'], cfg['e'], cfg['d'])
network = ofa_network.get_active_subnet()
print('Predicted accuracy: {} \nArthemetic Intensity of searched Model:{}'.format(result_lis[2][0]*100,1/result_lis[2][2]) )

Prepartion for training datase

In [ ]:
#lets train model in Imagenet 100k

batch_size = 32

train_path = '/kaggle/working/imagenet_subtrain'

train_data = datasets.ImageFolder(
            root= train_path,
            transform=build_val_transform(224)
        )

train_loader = torch.utils.data.DataLoader(
        train_data,
        batch_size=batch_size, 
        shuffle = True,
        num_workers=4,  
        pin_memory=True,
        drop_last=False,
    )

print('The ImageNet train subset is ready. Size : {}'.format(len(train_loader)*batch_size))


**Training Code to train a network **

In [ ]:

#Function for training the model, forware prop and backward prop 

def train_model(model, criterion, optimizer, scheduler, num_epochs=1):
    since = time.time()

    #storing epoch data
    epoch_data =     {
        'epoch': [],
        'train': {'loss': [], 'acc': []},
        'val': {'loss': [], 'acc': [] }
    }
    
    # Create a temporary directory in Kaggle's temp directory
    tempdir = '/kaggle/working/temp'
    os.makedirs(tempdir, exist_ok=True)
    best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

    torch.save(model.state_dict(), best_model_params_path)
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)
        epoch_data['epoch'].append(epoch+1)
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            running_loss = 0.0
            running_corrects = 0

            

            for inputs, labels in tqdm(dataloaders[phase], leave=False):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            epoch_data[phase]['loss'].append(epoch_loss)
            epoch_data[phase]['acc'].append(epoch_acc)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                torch.save(model.state_dict(), best_model_params_path)

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    model.load_state_dict(torch.load(best_model_params_path))

    # Clean up the temporary directory
    shutil.rmtree(tempdir)

    return model, epoch_data


In [ ]:
dataloaders = {}
dataloaders['train'] = train_loader
dataloaders['val'] = val_loader

dataset_sizes = {'train': len(train_loader)*32,
                'val': len(val_loader)*32}

**Lets try retraining the model with ops/byte > 20**

In [ ]:
device = 'cuda' if cuda_available else 'cpu'

model = network.to(device)
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
torch.cuda.empty_cache()
model, epoch_data = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=1)

#### It is clear that accuracy is recoverable and we can obtain comparable accuracy with our efficiency contraints by retraining for few epochs 

## 6. Model Conversion and Saving

In [ ]:
torch.save(model, '20.08ops_byte_79.pth')

In [ ]:
import torch.onnx 

#Function to Convert to ONNX 
def Convert_ONNX(model, in_shape, name): 

    # set the model to inference mode 
    model.eval() 

    # Let's create a dummy input tensor  

    dummy_input = torch.randn(in_shape, requires_grad=True)  

    # Export the model   
    torch.onnx.export(model,         # model being run 
         dummy_input,       # model input (or a tuple for multiple inputs) 
         f"{name}.onnx",       # where to save the model  
         export_params=True,  # store the trained parameter weights inside the model file 
         opset_version=10,    # the ONNX version to export the model to 
         do_constant_folding=True,  # whether to execute constant folding for optimization 
         input_names = ['modelInput'],   # the model's input names 
         output_names = ['modelOutput'], # the model's output names 
         dynamic_axes={'modelInput' : {0 : 'batch_size'},    # variable length axes 
                                'modelOutput' : {0 : 'batch_size'}}) 
    print(" ") 
    print('Model has been converted to ONNX')

In [ ]:
'ResNet-152':81.3,
'Efficientnet V2 Small': 83.6,